In [27]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import time
from unidecode import unidecode
import numpy as np
from itertools import chain

final_qb_data = pd.read_csv("final position group data/final_qb_data.csv")
final_qb_data['YearsBack'] = 2024 - final_qb_data['season'].astype(int)
final_qb_data.to_csv('final position group data/final_qb_data.csv')

def dfMaker():
     #if ppr is 0 it is non ppr, if it is 1 it is half ppr, if its 2 it is full ppr. loops through
    for ppr in range(3):
        
        #read csv files into pandas
        oldQBStats = pd.read_csv("final position group data/final_qb_data.csv")
        currTeamsRoster = pd.read_csv("Roster Creation/currYearRoster.csv")
        pastTeamsRoster = pd.read_csv("Roster Creation/teamsPastRoster.csv")
        currAVs = pd.read_csv("approximate value data/currAVs.csv")

        #make empty dfs and list to put players into
        completeDFQB = pd.DataFrame(columns= ["team", "position", "penalty", "GP", "player_display_name", "age", 'completions', 'attempts', 'passing_yards',
       'passing_tds', 'interceptions', 'sacks',
       'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch',
       'passing_first_downs', 'passing_2pt_conversions', 
       'carries', 'rushing_yards', 'rushing_tds',
       'rushing_fumbles_lost', 'rushing_first_downs',
       'rushing_2pt_conversions', "PPG"])
        rookieList = []

        f = 0

        for index in range(len(currTeamsRoster)):

            f+=1
            #boolean to break if rookie
            breakBool = False

            #years back variable
            x = 1

            #penalty variable for injuries
            penalty = 0

            #current players place to store information
            individualDFQB = pd.DataFrame(columns= ["team", "position", "penalty", "GP", "player_display_name", "age", 'completions', 'attempts', 'passing_yards',
       'passing_tds', 'interceptions', 'sacks',
       'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch',
       'passing_first_downs', 'passing_2pt_conversions', 'carries', 
       'rushing_yards', 'rushing_tds',
       'rushing_fumbles_lost', 'rushing_first_downs',
       'rushing_2pt_conversions', "PPG"])
            individualDFQB.loc[0] = [0] * len(individualDFQB.columns)

            #current players informaton
            year = currTeamsRoster.loc[index, 'Yrs']
            bday = currTeamsRoster.loc[index, 'BirthDate']
            pos = currTeamsRoster.loc[index, 'Pos']
            age = currTeamsRoster.loc[index, 'Age']
            name = currTeamsRoster.loc[index, 'Player']
            team = currTeamsRoster.loc[index, 'Team']

            games = 0

            #see if rookie
            if year == "Rook":
                #add player to rookie column
                dict = {}
                dict = {"Name": name, "Year": year, "Bday": bday, "Age": age, "Team": team}
                rookieList.append(dict)
                breakBool = True

                continue

            while True:
            
                #get all player rows from past year
                playerRowRoster = pastTeamsRoster.loc[(pastTeamsRoster['Player'] == name) & (pastTeamsRoster["YearsBack"] == x) & (pastTeamsRoster["BirthDate"] == bday)].copy()
                playerRowRoster = playerRowRoster.reset_index()
                
                #if there is no data for player it goes back to find data until years back (x) > 3
                if playerRowRoster.empty:
                    
                    if x>3:
                        if games>6:
                            #breaks if needed number of games is aqcuired.
                            break
                        elif (int(year)<4):
                            #add player to rookie column
                            dict = {}
                            dict = {"Name": name, "Year": year, "Bday": bday, "Age": age, "Team": team}
                            rookieList.append(dict)
                            breakBool = True

                            #breaks if cannot go back another year
                            break
                        else:
                            breakBool = True
                            break

                    x=x+1
                    continue
            
                for ind in range(len(playerRowRoster)):
                    teamCurr = playerRowRoster.loc[ind, "Team"]
                    numberCurr = playerRowRoster.loc[ind, "No."]
                    

                    if pos == "QB":
                        #gets correct player using team name, player name, and player number
                        playerRowStats = oldQBStats.loc[(oldQBStats["team"]== teamCurr) & (oldQBStats["player_display_name"]==name) & (oldQBStats["YearsBack"]==x)]
                        playerRowStats = playerRowStats.reset_index() 
                        playerRowStats = playerRowStats.fillna(0)

                        if playerRowStats.empty:
                            continue
                                                    
                        columns_to_update = [
                        'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions',
                        'sacks', 'sack_fumbles_lost', 'passing_air_yards',
                        'passing_yards_after_catch', 'passing_first_downs', 'passing_2pt_conversions',
                        'carries', 'rushing_yards', 'rushing_tds',
                        'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_2pt_conversions'
                        ]

                        for column in columns_to_update:
                            individualDFQB[column] = playerRowStats.loc[0, column] + individualDFQB[column]
    
                        
                        games = games + playerRowStats.loc[0, "GP"]


                        if (x==1):
                            penalty = penalty + playerRowStats.loc[0, "GP"]   
                        elif (x==2):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.9)   
                        elif (x==3):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.8)
                        elif (x==4):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.7) 
                        elif (x==5):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.6)
                        elif (x==6):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.5)
                        elif (x==7):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.4)
                        elif (x==8):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.3)
                        elif (x==9):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.2)
                        elif (x==10):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.1)   
                
                if (games<6) and (x>3) and (int(year)<4):
                    #add player to rookie column
                    dict = {}
                    dict = {"Name": name, "Year": year, "Bday": bday, "Age": age, "Team": team}
                    rookieList.append(dict)
                    breakBool = True
                    break
                elif (games<5) and (x>3):
                    breakBool = True
                    break
                if (games>6) or (x>5):
                    break

                x+=1

                        #break if rookie
            if (breakBool == True):
                continue

            if pos == "QB":
                #make it all per game not total
                columns_to_update = [
                        'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions',
                        'sacks', 'sack_fumbles_lost', 'passing_air_yards',
                        'passing_yards_after_catch', 'passing_first_downs', 'passing_2pt_conversions',
                        'carries', 'rushing_yards', 'rushing_tds',
                        'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_2pt_conversions'
                ]
                for column in columns_to_update:
                        individualDFQB[column] = individualDFQB[column] / games
                individualDFQB["player_display_name"] = name
                individualDFQB["position"] = pos
                individualDFQB["team"] = team
                individualDFQB["GP"] = games
                individualDFQB["penalty"] = penalty/games

                individualDFQB.loc[:, "PPG"] = (individualDFQB["rushing_yards"]*0.1) + (individualDFQB["passing_yards"]*.04) + (individualDFQB["rushing_tds"]*6) + (individualDFQB["passing_tds"]*4) + (individualDFQB["rushing_fumbles_lost"]*-2) + (individualDFQB["sack_fumbles_lost"]*-2) + (individualDFQB["interceptions"]*-2)
                individualDFQB.loc[:, "PPG"] = individualDFQB["PPG"]
                
                #add other columns needed
                individualDFQB["age"] = age

                completeDFQB = pd.concat([completeDFQB, individualDFQB], ignore_index=True, join="inner")  
        if ppr == 0:
            completeDFQB = pd.merge(completeDFQB, currAVs, on='team', how='left')
            completeDFQB = completeDFQB.sort_values(by=['PPG'])
            completeDFQB.to_csv("QBDFForModelNonPPR.csv", encoding='utf-8', index=False)
        elif ppr == 1:
            completeDFQB = pd.merge(completeDFQB, currAVs, on='team', how='left')
            completeDFQB = completeDFQB.sort_values(by=['PPG'])
            completeDFQB.to_csv("QBDFForModelHalfPPR.csv", encoding='utf-8', index=False)
        elif ppr == 2:
            completeDFQB = pd.merge(completeDFQB, currAVs, on='team', how='left')
            completeDFQB = completeDFQB.sort_values(by=['PPG'])
            completeDFQB.to_csv("QBDFForModelPPR.csv", encoding='utf-8', index=False)

dfMaker()


In [28]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import time
from unidecode import unidecode
import numpy as np
from itertools import chain

final_rb_data = pd.read_csv("final position group data/final_rb_data.csv")
final_rb_data['YearsBack'] = 2024 - final_rb_data['season'].astype(int)
final_rb_data.to_csv('final position group data/final_rb_data.csv')

def dfMaker():
     #if ppr is 0 it is non ppr, if it is 1 it is half ppr, if its 2 it is full ppr. loops through
    for ppr in range(3):
        
        #read csv files into pandas
        oldRBStats = pd.read_csv("final position group data/final_rb_data.csv")
        currTeamsRoster = pd.read_csv("Roster Creation/currYearRoster.csv")
        pastTeamsRoster = pd.read_csv("Roster Creation/teamsPastRoster.csv")
        currAVs = pd.read_csv("approximate value data/currAVs.csv")

        #make empty dfs and list to put players into
        completeDFRB= pd.DataFrame(columns= ["team", "position", "penalty", "GP", "player_display_name", "age", 'carries', 'rushing_yards', 'rushing_tds',
       'rushing_fumbles_lost', 'rushing_first_downs',
       'rushing_2pt_conversions', 'receptions', 'targets',
       'receiving_yards', 'receiving_tds',
       'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs',
       'receiving_2pt_conversions', 'special_teams_tds',
       'rrtd', "PPG"])
        rookieList = []

        f = 0

        for index in range(len(currTeamsRoster)):

            f+=1
            #boolean to break if rookie
            breakBool = False

            #years back variable
            x = 1

            #penalty variable for injuries
            penalty = 0

            #current players place to store information
            individualDFRB= pd.DataFrame(columns= ["team", "position", "penalty", "GP", "player_display_name", "age", 'carries', 'rushing_yards', 'rushing_tds',
       'rushing_fumbles_lost', 'rushing_first_downs',
       'rushing_2pt_conversions', 'receptions', 'targets',
       'receiving_yards', 'receiving_tds',
       'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs',
       'receiving_2pt_conversions', 'special_teams_tds',
       'rrtd', "PPG"])
            individualDFRB.loc[0] = [0] * len(individualDFRB.columns)

            #current players informaton
            year = currTeamsRoster.loc[index, 'Yrs']
            bday = currTeamsRoster.loc[index, 'BirthDate']
            pos = currTeamsRoster.loc[index, 'Pos']
            age = currTeamsRoster.loc[index, 'Age']
            name = currTeamsRoster.loc[index, 'Player']
            team = currTeamsRoster.loc[index, 'Team']

            games = 0

            #see if rookie
            if year == "Rook":
                #add player to rookie column
                dict = {}
                dict = {"Name": name, "Year": year, "Bday": bday, "Age": age, "Team": team}
                rookieList.append(dict)
                breakBool = True

                continue

            while True:
            
                #get all player rows from past year
                playerRowRoster = pastTeamsRoster.loc[(pastTeamsRoster['Player'] == name) & (pastTeamsRoster["YearsBack"] == x) & (pastTeamsRoster["BirthDate"] == bday)].copy()
                playerRowRoster = playerRowRoster.reset_index()
                
                #if there is no data for player it goes back to find data until years back (x) > 3
                if playerRowRoster.empty:
                    
                    if x>3:
                        if games>6:
                            #breaks if needed number of games is aqcuired.
                            break
                        elif (int(year)<4):
                            #add player to rookie column
                            dict = {}
                            dict = {"Name": name, "Year": year, "Bday": bday, "Age": age, "Team": team}
                            rookieList.append(dict)
                            breakBool = True

                            #breaks if cannot go back another year
                            break
                        else:
                            breakBool = True
                            break

                    x=x+1
                    continue
            
                for ind in range(len(playerRowRoster)):
                    teamCurr = playerRowRoster.loc[ind, "Team"]
                    numberCurr = playerRowRoster.loc[ind, "No."]
                    

                    if pos == "RB":
                        #gets correct player using team name, player name, and player number
                        playerRowStats = oldRBStats.loc[(oldRBStats["team"]== teamCurr) & (oldRBStats["player_display_name"]==name) & (oldRBStats["YearsBack"]==x)]
                        playerRowStats = playerRowStats.reset_index() 
                        playerRowStats = playerRowStats.fillna(0)

                        if playerRowStats.empty:
                            continue
                                                    
                        columns_to_update = ['carries', 'rushing_yards', 'rushing_tds',
                        'rushing_fumbles_lost', 'rushing_first_downs',
                        'rushing_2pt_conversions', 'receptions', 'targets',
                        'receiving_yards', 'receiving_tds',
                        'receiving_fumbles_lost', 'receiving_air_yards',
                        'receiving_yards_after_catch', 'receiving_first_downs',
                        'receiving_2pt_conversions','special_teams_tds',
                        'rrtd']

                        for column in columns_to_update:
                            individualDFRB[column] = playerRowStats.loc[0, column] + individualDFRB[column]

                        
                        games = games + playerRowStats.loc[0, "GP"]


                        if (x==1):
                            penalty = penalty + playerRowStats.loc[0, "GP"]   
                        elif (x==2):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.9)   
                        elif (x==3):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.8)
                        elif (x==4):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.7) 
                        elif (x==5):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.6)
                        elif (x==6):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.5)
                        elif (x==7):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.4)
                        elif (x==8):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.3)
                        elif (x==9):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.2)
                        elif (x==10):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.1)    
                
                if (games<6) and (x>3) and (int(year)<4):
                    #add player to rookie column
                    dict = {}
                    dict = {"Name": name, "Year": year, "Bday": bday, "Age": age, "Team": team}
                    rookieList.append(dict)
                    breakBool = True
                    break
                elif (games<6) and (x>3):
                    breakBool = True
                    break
                if (games>6) or (x>3):
                    break

                x+=1

                        #break if rookie
            if (breakBool == True):
                continue

            if pos == "RB":
                #make it all per game not total
                columns_to_update = ['carries', 'rushing_yards', 'rushing_tds',
                        'rushing_fumbles_lost', 'rushing_first_downs',
                        'rushing_2pt_conversions', 'receptions', 'targets',
                        'receiving_yards', 'receiving_tds',
                        'receiving_fumbles_lost', 'receiving_air_yards',
                        'receiving_yards_after_catch', 'receiving_first_downs',
                        'receiving_2pt_conversions','special_teams_tds',
                        'rrtd']
                for column in columns_to_update:
                        individualDFRB[column] = individualDFRB[column] / games
                individualDFRB["player_display_name"] = name
                individualDFRB["position"] = pos
                individualDFRB["team"] = team
                individualDFRB["GP"] = games
                individualDFRB["penalty"] = penalty/games

                individualDFRB.loc[:, "PPG"] = (individualDFRB["rushing_yards"]*0.1)+ (individualDFRB["rushing_tds"]*6)+ (individualDFRB["rushing_fumbles_lost"]*-2) + (individualDFRB["receiving_yards"]*0.1) + (individualDFRB["receiving_tds"]*6) + (individualDFRB["receiving_fumbles_lost"]*-2)
                if ppr == 2:
                    individualDFRB.loc[:, "PPG"] = individualDFRB["PPG"] + (individualDFRB["receptions"])
                elif ppr == 1:
                    individualDFRB.loc[:, "PPG"] = individualDFRB["PPG"] + ((individualDFRB["receptions"])/2)
                else:
                    individualDFRB.loc[:, "PPG"] = individualDFRB["PPG"]
                
                #add other columns needed
                individualDFRB["age"] = age

                completeDFRB = pd.concat([completeDFRB, individualDFRB], ignore_index=True, join="inner")  
        if ppr == 0:
            completeDFRB = pd.merge(completeDFRB, currAVs, on='team', how='left')
            completeDFRB = completeDFRB.sort_values(by=['PPG'])
            completeDFRB.to_csv("RBDFForModelNonPPR.csv", encoding='utf-8', index=False)
        elif ppr == 1:
            completeDFRB = pd.merge(completeDFRB, currAVs, on='team', how='left')
            completeDFRB = completeDFRB.sort_values(by=['PPG'])
            completeDFRB.to_csv("RBDFForModelHalfPPR.csv", encoding='utf-8', index=False)
        elif ppr == 2:
            completeDFRB = pd.merge(completeDFRB, currAVs, on='team', how='left')
            completeDFRB = completeDFRB.sort_values(by=['PPG'])
            completeDFRB.to_csv("RBDFForModelPPR.csv", encoding='utf-8', index=False)

dfMaker()


In [29]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import time
from unidecode import unidecode
import numpy as np
from itertools import chain

final_wrte_data = pd.read_csv("final position group data/final_wrte_data.csv")
final_wrte_data['YearsBack'] = 2024 - final_wrte_data['season'].astype(int)
final_wrte_data.to_csv('final position group data/final_wrte_data.csv')

def dfMaker():
     #if ppr is 0 it is non ppr, if it is 1 it is half ppr, if its 2 it is full ppr. loops through
    for ppr in range(3):
        
        #read csv files into pandas
        oldWRTEStats = pd.read_csv("final position group data/final_wrte_data.csv")
        currTeamsRoster = pd.read_csv("Roster Creation/currYearRoster.csv")
        pastTeamsRoster = pd.read_csv("Roster Creation/teamsPastRoster.csv")
        currAVs = pd.read_csv("approximate value data/currAVs.csv")

        #make empty dfs and list to put players into
        completeDFWRTE= pd.DataFrame(columns= ["team", "position", "penalty", "GP", "player_display_name", "age", 'carries', 'rushing_yards', 'rushing_tds',
       'rushing_fumbles_lost', 'rushing_first_downs',
       'rushing_2pt_conversions', 'receptions', 'targets',
       'receiving_yards', 'receiving_tds',
       'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs',
       'receiving_2pt_conversions', 'special_teams_tds',
       'rrtd', "PPG"])
        rookieList = []

        f = 0

        for index in range(len(currTeamsRoster)):

            f+=1
            #boolean to break if rookie
            breakBool = False

            #years back variable
            x = 1

            #penalty variable for injuries
            penalty = 0

            #current players place to store information
            individualDFWRTE= pd.DataFrame(columns= ["team", "position", "penalty", "GP", "player_display_name", "age", 'carries', 'rushing_yards', 'rushing_tds',
       'rushing_fumbles_lost', 'rushing_first_downs',
       'rushing_2pt_conversions', 'receptions', 'targets',
       'receiving_yards', 'receiving_tds',
       'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs',
       'receiving_2pt_conversions', 'special_teams_tds',
       'rrtd', "PPG"])
            individualDFWRTE.loc[0] = [0] * len(individualDFWRTE.columns)

            #current players informaton
            year = currTeamsRoster.loc[index, 'Yrs']
            bday = currTeamsRoster.loc[index, 'BirthDate']
            pos = currTeamsRoster.loc[index, 'Pos']
            age = currTeamsRoster.loc[index, 'Age']
            name = currTeamsRoster.loc[index, 'Player']
            team = currTeamsRoster.loc[index, 'Team']

            games = 0

            #see if rookie
            if year == "Rook":
                #add player to rookie column
                dict = {}
                dict = {"Name": name, "Year": year, "Bday": bday, "Age": age, "Team": team}
                rookieList.append(dict)
                breakBool = True

                continue

            while True:
            
                #get all player rows from past year
                playerRowRoster = pastTeamsRoster.loc[(pastTeamsRoster['Player'] == name) & (pastTeamsRoster["YearsBack"] == x) & (pastTeamsRoster["BirthDate"] == bday)].copy()
                playerRowRoster = playerRowRoster.reset_index()
                
                #if there is no data for player it goes back to find data until years back (x) > 3
                if playerRowRoster.empty:
                    
                    if x>3:
                        if games>6:
                            #breaks if needed number of games is aqcuired.
                            break
                        elif (int(year)<4):
                            #add player to rookie column
                            dict = {}
                            dict = {"Name": name, "Year": year, "Bday": bday, "Age": age, "Team": team}
                            rookieList.append(dict)
                            breakBool = True

                            #breaks if cannot go back another year
                            break
                        else:
                            breakBool = True
                            break

                    x=x+1
                    continue
            
                for ind in range(len(playerRowRoster)):
                    teamCurr = playerRowRoster.loc[ind, "Team"]
                    numberCurr = playerRowRoster.loc[ind, "No."]
                    

                    if pos in ("WR", "TE"):
                        #gets correct player using team name, player name, and player number
                        playerRowStats = oldWRTEStats.loc[(oldWRTEStats["team"]== teamCurr) & (oldWRTEStats["player_display_name"]==name) & (oldWRTEStats["YearsBack"]==x)]
                        playerRowStats = playerRowStats.reset_index() 
                        playerRowStats = playerRowStats.fillna(0)

                        if playerRowStats.empty:
                            continue
                                                    
                        columns_to_update = ['carries', 'rushing_yards', 'rushing_tds',
                        'rushing_fumbles_lost', 'rushing_first_downs',
                        'rushing_2pt_conversions', 'receptions', 'targets',
                        'receiving_yards', 'receiving_tds',
                        'receiving_fumbles_lost', 'receiving_air_yards',
                        'receiving_yards_after_catch', 'receiving_first_downs',
                        'receiving_2pt_conversions','special_teams_tds',
                        'rrtd']

                        for column in columns_to_update:
                            individualDFWRTE[column] = playerRowStats.loc[0, column] + individualDFWRTE[column]

                        
                        games = games + playerRowStats.loc[0, "GP"]


                        if (x==1):
                            penalty = penalty + playerRowStats.loc[0, "GP"]   
                        elif (x==2):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.9)   
                        elif (x==3):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.8)
                        elif (x==4):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.7) 
                        elif (x==5):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.6)
                        elif (x==6):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.5)
                        elif (x==7):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.4)
                        elif (x==8):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.3)
                        elif (x==9):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.2)
                        elif (x==10):
                            penalty = penalty + (playerRowStats.loc[0, "GP"]*0.1)    
                
                if (games<6) and (x>3) and (int(year)<4):
                    #add player to rookie column
                    dict = {}
                    dict = {"Name": name, "Year": year, "Bday": bday, "Age": age, "Team": team}
                    rookieList.append(dict)
                    breakBool = True
                    break
                elif (games<6) and (x>3):
                    breakBool = True
                    break
                if (games>6) or (x>3):
                    break

                x+=1

                        #break if rookie
            if (breakBool == True):
                continue

            if pos in ("WR", "TE"):
                #make it all per game not total
                columns_to_update = ['carries', 'rushing_yards', 'rushing_tds',
                        'rushing_fumbles_lost', 'rushing_first_downs',
                        'rushing_2pt_conversions', 'receptions', 'targets',
                        'receiving_yards', 'receiving_tds',
                        'receiving_fumbles_lost', 'receiving_air_yards',
                        'receiving_yards_after_catch', 'receiving_first_downs',
                        'receiving_2pt_conversions','special_teams_tds',
                        'rrtd']
                for column in columns_to_update:
                        individualDFWRTE[column] = individualDFWRTE[column] / games
                individualDFWRTE["player_display_name"] = name
                individualDFWRTE["position"] = pos
                individualDFWRTE["team"] = team
                individualDFWRTE["GP"] = games
                individualDFWRTE["penalty"] = penalty/games

                individualDFWRTE.loc[:, "PPG"] = (individualDFWRTE["rushing_yards"]*0.1)+ (individualDFWRTE["rushing_tds"]*6)+ (individualDFWRTE["rushing_fumbles_lost"]*-2) + (individualDFWRTE["receiving_yards"]*0.1) + (individualDFWRTE["receiving_tds"]*6) + (individualDFWRTE["receiving_fumbles_lost"]*-2)
                if ppr == 2:
                    individualDFWRTE.loc[:, "PPG"] = individualDFWRTE["PPG"] + (individualDFWRTE["receptions"])
                elif ppr == 1:
                    individualDFWRTE.loc[:, "PPG"] = individualDFWRTE["PPG"] + ((individualDFWRTE["receptions"])/2)
                else:
                    individualDFWRTE.loc[:, "PPG"] = individualDFWRTE["PPG"]
                
                #add other columns needed
                individualDFWRTE["age"] = age

                completeDFWRTE = pd.concat([completeDFWRTE, individualDFWRTE], ignore_index=True, join="inner")  
        if ppr == 0:
            completeDFWRTE = pd.merge(completeDFWRTE, currAVs, on='team', how='left')
            completeDFWRTE = completeDFWRTE.sort_values(by=['PPG'])
            completeDFWRTE.to_csv("WRTEDFForModelNonPPR.csv", encoding='utf-8', index=False)
        elif ppr == 1:
            completeDFWRTE = pd.merge(completeDFWRTE, currAVs, on='team', how='left')
            completeDFWRTE = completeDFWRTE.sort_values(by=['PPG'])
            completeDFWRTE.to_csv("WRTEDFForModelHalfPPR.csv", encoding='utf-8', index=False)
        elif ppr == 2:
            completeDFWRTE = pd.merge(completeDFWRTE, currAVs, on='team', how='left')
            completeDFWRTE = completeDFWRTE.sort_values(by=['PPG'])
            completeDFWRTE.to_csv("WRTEDFForModelPPR.csv", encoding='utf-8', index=False)

dfMaker()


In [30]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import time
from unidecode import unidecode
import numpy as np
from itertools import chain
import joblib
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore", message="X has feature names, but GradientBoostingRegressor was fitted without feature names")

#gets ppg back into normal form
def getScaleBack(df):
  #index of column
  column_index = df.columns.get_loc("PPG")

  #min value of column:
  min_value = df["PPG"].min()

  #scaling valye of column
  #scaling_factor = scaler.scale_[column_index]
  max_value = df["PPG"].max()

  #array to be used later to scale each data
  arr = [min_value, max_value]

  return arr

def scorer():

    #scaler to scale data
    scaler = MinMaxScaler()

    #if 0 its non ppr, if 1 its half ppr, if 2 its full ppr. loops through.

    for ppr in range(3):
        #dict of scores
        dictScores = {}

        if ppr == 0:
            #read in player stats df and make positional dfs
            qb = pd.read_csv("QBDFForModelNonPPR.csv")
            rb = pd.read_csv("RBDFForModelNonPPR.csv")
            wrte = pd.read_csv("WRTEDFForModelNonPPR.csv")

            #loads models
            rbModel = joblib.load("rb models/rbModelNonPPR.joblib")
            wrteModel = joblib.load("wrte models/wrteModelNonPPR.joblib")
            qbModel = joblib.load("qb models/qbModelNonPPR.joblib")
        elif ppr == 1:
            #read in player stats df and make positional dfs
            qb = pd.read_csv("QBDFForModelHalfPPR.csv")
            rb = pd.read_csv("RBDFForModelHalfPPR.csv")
            wrte = pd.read_csv("WRTEDFForModelHalfPPR.csv")

            #loads models
            rbModel = joblib.load("rb models/rbModelHalfPPR.joblib")
            wrteModel = joblib.load("wrte models/wrteModelHalfPPR.joblib")
            qbModel = joblib.load("qb models/qbModelHalfPPR.joblib")
        elif ppr == 2:        
            #read in player stats df and make positional dfs
            qb = pd.read_csv("QBDFForModelPPR.csv")
            rb = pd.read_csv("RBDFForModelPPR.csv")
            wrte = pd.read_csv("WRTEDFForModelPPR.csv")

            #loads models
            rbModel = joblib.load("rb models/rbModelPPR.joblib")
            wrteModel = joblib.load("wrte models/wrteModelPPR.joblib")
            qbModel = joblib.load("qb models/qbModelPPR.joblib")

        #dictionary to easily call models
        modelsDict = {"QB": qbModel, "WRTE": wrteModel, "RB": rbModel,}

        #gets fantasypoints scale per each position
        scaleQB = getScaleBack(qb)
        scaleRB = getScaleBack(rb)
        scaleWRTE = getScaleBack(wrte)

        #create a scaled version of each positional df, and have it be same parameters as models
        rbScaled = rb.copy()
        rbScaled = rbScaled.drop(columns=["GP", "player_display_name", "team", "position", "penalty", "PPG"])        
        rbScaled[rbScaled.columns] = scaler.fit_transform(rbScaled[rbScaled.columns])

        wrteScaled = wrte.copy()
        wrteScaled = wrteScaled.drop(columns=["GP","player_display_name", "team", "position", "penalty", "PPG"])        
        wrteScaled[wrteScaled.columns] = scaler.fit_transform(wrteScaled[wrteScaled.columns])
        
        qbScaled = qb.copy()
        qbScaled = qbScaled.drop(columns=["GP", "player_display_name", "team", "position", "penalty", "PPG"])        
        qbScaled[qbScaled.columns] = scaler.fit_transform(qbScaled[qbScaled.columns])

        #have these so it can iterate through easily
        allPosDFs = [rb, wrte, qb]
        allPosDfsScaled = [rbScaled, wrteScaled, qbScaled]
        scaleBack = [scaleRB, scaleWRTE, scaleQB]

        currPosDict = {}
        indPosArr = []

        #iterates through each position
        for ind in range(len(allPosDFs)):
            #gets current position df, scaled current df, and array to help get ppg to normal.
            currDF = allPosDFs[ind]
            scaled = allPosDfsScaled[ind]
            arr = scaleBack[ind]

            #dictionary to store current position grades.
            # currPosDict = {}
            currDF["Final PPG"] = 0
        
            #iterates through all players in current position.
            for i in range(len(currDF)):

                #gets current information for current player
                currRow = currDF.iloc[[i]]
                currRow = currRow.reset_index()
                scaled = scaled.reset_index()
                scaled = scaled.drop(columns=["index"])
                currRowForModel = scaled.iloc[[i]] 

                pos = currRow.loc[0, "position"]
                name = currRow.loc[0, "player_display_name"]
                team = currRow.loc[0, "team"]
                penalty = currRow.loc[0, "penalty"]

                if pos in ["WR", "TE"]:
                    pos = "WRTE"   

                #gets in correct positinal model and predicts each player
                model = modelsDict[pos]
                prediction = model.predict(currRowForModel)  

                #inverse transform the scaled predictions to get the original scale by reversing formula
                prediction = (prediction*(arr[1] - arr[0])) + arr[0]

                #this is where the penalty is added and prediction is entered into the dictionary
                currDF.at[i, "Final PPG"] = prediction[0]  

            #appends current position grades to all of them.
            indPosArr.append(currDF)

            
    
        #sorts dictionary
        finalrbs = indPosArr[0].sort_values(by="Final PPG", ascending=False)
        finalwrte = indPosArr[1].sort_values(by="Final PPG", ascending=False)
        finalqbs = indPosArr[2].sort_values(by="Final PPG", ascending=False)

        finalrbs["Rank"] = np.arange(1, len(finalrbs) + 1)
        finalwrte["Rank"] = np.arange(1, len(finalwrte) + 1)
        finalqbs["Rank"] = np.arange(1, len(finalqbs) + 1)

        #selecting necessary columns
        finalrbs = finalrbs[["Rank", "player_display_name", "team", "position", "Final PPG"]]
        finalwrte = finalwrte[["Rank", "player_display_name", "team", "position", "Final PPG"]]
        finalqbs = finalqbs[["Rank", "player_display_name", "team", "position", "Final PPG"]]

        #renaming columns
        finalrbs.columns = ["Rank", "Name", "Team", "Position", "Final PPG"]
        finalwrte.columns = ["Rank", "Name", "Team", "Position", "Final PPG"]
        finalqbs.columns = ["Rank", "Name", "Team", "Position", "Final PPG"]


        #writes into csv files
        if ppr == 0:
            finalrbs.to_csv("Final Rankings/RBs_NonPPR.csv", encoding='utf-8', index=False)
            finalwrte.to_csv("Final Rankings/WRTE_NonPPR.csv", encoding='utf-8', index=False)
            finalqbs.to_csv("Final Rankings/QBs_NonPPR.csv", encoding='utf-8', index=False)
        elif ppr == 1:
            finalrbs.to_csv("Final Rankings/RBs_HalfPPR.csv", encoding='utf-8', index=False)
            finalwrte.to_csv("Final Rankings/WRTE_HalfPPR.csv", encoding='utf-8', index=False)
            finalqbs.to_csv("Final Rankings/QBs_HalfPPR.csv", encoding='utf-8', index=False)
        elif ppr == 2:
            finalrbs.to_csv("Final Rankings/RBs_PPR.csv", encoding='utf-8', index=False)
            finalwrte.to_csv("Final Rankings/WRTE_PPR.csv", encoding='utf-8', index=False)
            finalqbs.to_csv("Final Rankings/QBs_PPR.csv", encoding='utf-8', index=False)



scorer()